In [1]:
#Directory Structure
#~/Documents/TAQData <--- has the parquet files
#~/gitcode/AmitG/TAQCode <--- has the jupypter and python code


In [2]:
import pyarrow.parquet as pq

# ---- quotes file ----
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
pf_quotes = pq.ParquetFile(quotes_path)
print("🧾 Quotes schema:")
print(pf_quotes.schema_arrow)     # column names and types only

# ---- trades file ----
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'
pf_trades = pq.ParquetFile(trades_path)
print("\n💹 Trades schema:")
print(pf_trades.schema_arrow)


🧾 Quotes schema:
DATE: date32[day]
TIME_M: time64[ns]
EX: large_string
BID: double
BIDSIZ: int64
ASK: double
ASKSIZ: int64
QU_COND: large_string
QU_SEQNUM: int64
NATBBO_IND: large_string
QU_CANCEL: large_string
QU_SOURCE: large_string
SYM_ROOT: large_string
SYM_SUFFIX: large_string

💹 Trades schema:
date: date32[day]
time_m: time64[us]
time_m_nano: int16
ex: string
sym_root: string
sym_suffix: string
tr_scond: string
size: int32
price: double
tr_stop_ind: string
tr_corr: string
tr_seqnum: int64
tr_id: string
tr_source: string
tr_rf: string
part_time: time64[us]
part_time_nano: int16
trf_time: time64[us]
trf_time_nano: int16
tte_ind: string


In [3]:
import pyarrow.parquet as pq

# Paths to your files
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'

# Efficiently count rows without loading data
quotes_rows = pq.ParquetFile(quotes_path).metadata.num_rows
trades_rows = pq.ParquetFile(trades_path).metadata.num_rows

print(f"Quotes rows: {quotes_rows:,}")
print(f"Trades rows: {trades_rows:,}")


Quotes rows: 1,957,290,808
Trades rows: 89,518,138


In [4]:
import pyarrow.parquet as pq
import os

DATA_DIR = "~/Documents/TAQData"
quotes_path = os.path.expanduser(f"{DATA_DIR}/data_quotes_2023_12_15.parquet")
trades_path = os.path.expanduser(f"{DATA_DIR}/data_trades_2023_12_15.parquet")

def parquet_date_range(path, date_col):
    pf = pq.ParquetFile(path)
    for rg in range(pf.num_row_groups):
        df = pf.read_row_group(rg, columns=[date_col]).to_pandas()
        if not df.empty:
            dmin, dmax = df[date_col].min(), df[date_col].max()
            return dmin, dmax
    return None, None

quote_min, quote_max = parquet_date_range(quotes_path, "DATE")
trade_min, trade_max = parquet_date_range(trades_path, "date")

print(f"Quotes date range: {quote_min} → {quote_max}")
print(f"Trades date range: {trade_min} → {trade_max}")


Quotes date range: 2023-12-15 → 2023-12-15
Trades date range: 2023-12-15 → 2023-12-15


In [1]:
# 1) What symbols are actually present? (first ~20 row groups, TRADES)
import pyarrow.parquet as pq, pandas as pd, os
TRADES_FILE = os.path.expanduser("~/Documents/TAQData/data_trades_2023_12_15.parquet")
pf = pq.ParquetFile(TRADES_FILE)

seen = []
for rg in range(min(20, pf.num_row_groups)):
    df = pf.read_row_group(rg, columns=["sym_root","sym_suffix"]).to_pandas()
    df["sym_root"]  = df["sym_root"].astype(str).str.strip().str.upper()
    df["sym_suffix"]= df["sym_suffix"].astype(str).fillna("").str.strip().str.upper()
    seen.append(df.groupby(["sym_root","sym_suffix"]).size().reset_index(name="cnt"))
present = pd.concat(seen).groupby(["sym_root","sym_suffix"])["cnt"].sum().reset_index().sort_values("cnt", ascending=False)
print(present.head(25))


    sym_root sym_suffix     cnt
24      AAPL       NONE  625486
122     ADBE       NONE  140747
1         AA       NONE  128766
15       AAL       NONE   94629
52       ABT       NONE   77437
45      ABNB       NONE   76272
133      ADI       NONE   73992
32      ABBV       NONE   69560
165      AEE       NONE   68983
136      ADM       NONE   50396
91       ACN       NONE   48587
0          A       NONE   46061
75      ACGL       NONE   45061
150     ADSK       NONE   43176
180      AEO       NONE   42705
144      ADP       NONE   38899
176      AEM       NONE   35980
47       ABR       NONE   35705
19      AAOI       NONE   35294
10      AADI       NONE   32126
183      AEP       NONE   30936
21       AAP       NONE   28638
61      ACAD       NONE   26610
79      ACHR       NONE   26189
82       ACI       NONE   24420


In [12]:
#polar_try_partitioned.py for quotes and trades
import polars as pl
pl.Config.set_tbl_cols(50) 
pl.Config.set_tbl_width_chars(200)
pl.Config.set_tbl_rows(200) #polars.Config.tbl_rows = 50
#rename.py shall rename the cols of trades to uppercase and save since easier for joining

polars.config.Config

In [6]:
import polars as pl
import os

# Define the directory and pattern
directory = "../TAQData/processed_output_trades_upper"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'TIME_M_NANO': Int16, 'EX': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String, 'TR_SCOND': String, 'SIZE': Int32, 'PRICE': Float64, 'TR_STOP_IND': String, 'TR_CORR': String, 'TR_SEQNUM': Int64, 'TR_ID': String, 'TR_SOURCE': String, 'TR_RF': String, 'PART_TIME': Time, 'PART_TIME_NANO': Int16, 'TRF_TIME': Time, 'TRF_TIME_NANO': Int16, 'TTE_IND': String})


In [14]:
print(lazy_df.filter(pl.col("SYM_ROOT") == "AAPL").head(10).collect())

shape: (10, 14)
┌────────────┬──────────────────┬─────┬────────┬───┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M           ┆ EX  ┆ BID    ┆ … ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---              ┆ --- ┆ ---    ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time             ┆ str ┆ f64    ┆   ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪══════════════════╪═════╪════════╪═══╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.1149253 ┆ Z   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 77               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 03:59:00.1854122 ┆ K   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 94               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 04:00:00.0044523 ┆ K   ┆ 197.53 ┆ … ┆ null      ┆ N         

In [7]:
import polars as pl
import os

# Define the directory and pattern
directory = "../TAQData/processed_output_quotes"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'EX': String, 'BID': Float64, 'BIDSIZ': Int64, 'ASK': Float64, 'ASKSIZ': Int64, 'QU_COND': String, 'QU_SEQNUM': Int64, 'NATBBO_IND': String, 'QU_CANCEL': String, 'QU_SOURCE': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String})


In [8]:
print(lazy_df.filter((pl.col('SYM_ROOT') == 'AAPL') & ( pl.col('BIDSIZ').is_not_null() )).head(10).collect() )

shape: (10, 14)
┌────────────┬────────────────────┬─────┬────────┬────────┬────────┬────────┬─────────┬───────────┬────────────┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M             ┆ EX  ┆ BID    ┆ BIDSIZ ┆ ASK    ┆ ASKSIZ ┆ QU_COND ┆ QU_SEQNUM ┆ NATBBO_IND ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---                ┆ --- ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---     ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time               ┆ str ┆ f64    ┆ i64    ┆ f64    ┆ i64    ┆ str     ┆ i64       ┆ str        ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪════════════════════╪═════╪════════╪════════╪════════╪════════╪═════════╪═══════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.114925377 ┆ Z   ┆ 0.0    ┆ 0      ┆ 0.0    ┆ 0      ┆ L       ┆ 1103      ┆ 1          ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│ 2023-12-15 ┆ 03:

In [1]:
#run query.py for joint dataset
#run stats.py for top 50

In [29]:
#See the output of stats
import polars as pl

# Define the paths to your files
quotes_file = "stats_out_simple/top50_quotes_by_rows.csv"
trades_file = "stats_out_simple/top50_trades_by_volume.csv"

# Load both files lazily
lazy_quotes_df = pl.scan_csv(quotes_file)
lazy_trades_df = pl.scan_csv(trades_file)

# 1. Sum QUOTE_ROWS from the quotes file
quote_rows_sum = (
    lazy_quotes_df
    .select(pl.col("QUOTES_ROWS").sum().alias("TOTAL_QUOTE_ROWS"))
    .collect()
)

# 2. Sum TRADE_ROWS from the trades file
trade_rows_sum = (
    lazy_trades_df
    .select(pl.col("TRADES_ROWS").sum().alias("TOTAL_TRADE_ROWS"))
    .collect()
)

# Optional: Print the results
print("--- Quote Rows Sum ---")
print(quote_rows_sum)
print("\n--- Trade Rows Sum ---")
print(trade_rows_sum)

--- Quote Rows Sum ---
shape: (1, 1)
┌──────────────────┐
│ TOTAL_QUOTE_ROWS │
│ ---              │
│ i64              │
╞══════════════════╡
│ 328673090        │
└──────────────────┘

--- Trade Rows Sum ---
shape: (1, 1)
┌──────────────────┐
│ TOTAL_TRADE_ROWS │
│ ---              │
│ i64              │
╞══════════════════╡
│ 9324631          │
└──────────────────┘


In [25]:
TRADES_OUT_CSV = OUT_DIR / "top50_trades_by_volume.csv"
QUOTES_OUT_CSV = OUT_DIR / "top50_quotes_by_rows.csv"

print(f"Reading {TRADES_OUT_CSV} and {QUOTES_OUT_CSV} …")
trades_df = pl.read_csv(TRADES_OUT_CSV)
quotes_df = pl.read_csv(QUOTES_OUT_CSV)

print(trades_df.sort("TRADES_ROWS", descending=True))
print(quotes_df.sort("QUOTES_ROWS", descending=True))

Reading stats_out_simple/top50_trades_by_volume.csv and stats_out_simple/top50_quotes_by_rows.csv …
shape: (50, 4)
┌──────────┬────────────┬─────────────┬────────────┐
│ SYM_ROOT ┆ SYM_SUFFIX ┆ TRADES_ROWS ┆ TRADES_VOL │
│ ---      ┆ ---        ┆ ---         ┆ ---        │
│ str      ┆ str        ┆ i64         ┆ i64        │
╞══════════╪════════════╪═════════════╪════════════╡
│ TSLA     ┆            ┆ 1185389     ┆ 159068358  │
│ SPY      ┆            ┆ 669221      ┆ 159259628  │
│ AAPL     ┆            ┆ 625486      ┆ 183784854  │
│ AMZN     ┆            ┆ 492794      ┆ 156092695  │
│ UBER     ┆            ┆ 468538      ┆ 1145449108 │
│ MSFT     ┆            ┆ 447207      ┆ 111464305  │
│ CCCC     ┆            ┆ 339905      ┆ 85960043   │
│ INTC     ┆            ┆ 337153      ┆ 114212069  │
│ PFE      ┆            ┆ 301662      ┆ 287715357  │
│ PLTR     ┆            ┆ 278350      ┆ 130424480  │
│ GOOG     ┆            ┆ 266240      ┆ 87741515   │
│ BAC      ┆            ┆ 242610     

In [23]:
# join_trades_quotes.py — combine stats_simple outputs
import polars as pl
from pathlib import Path

OUT_DIR = Path("./stats_out_simple")
TRADES_OUT_CSV = OUT_DIR / "top50_trades_by_volume.csv"
QUOTES_OUT_CSV = OUT_DIR / "top50_quotes_by_rows.csv"
MERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"

print(f"Reading {TRADES_OUT_CSV} and {QUOTES_OUT_CSV} …")
trades_df = pl.read_csv(TRADES_OUT_CSV)
quotes_df = pl.read_csv(QUOTES_OUT_CSV)

# Join and sort descending by TRADES_VOL
merged = (
    trades_df
    .join(quotes_df, on=["SYM_ROOT", "SYM_SUFFIX"], how="outer")
#    .with_columns([
#        pl.col("TRADES_VOL").fill_null(0),
#        pl.col("TRADES_ROWS").fill_null(0),
#        pl.col("QUOTES_ROWS").fill_null(0)
#    ])
    .filter(pl.col("QUOTES_ROWS").is_not_null())
    .filter(pl.col("TRADES_VOL").is_not_null())
    .sort("TRADES_ROWS", descending=True)
    
)

print("\nMerged:")
print(merged)

merged.write_csv(MERGED_OUT_CSV)
print(f"\n✅ Combined file saved to → {MERGED_OUT_CSV}")


Reading stats_out_simple/top50_trades_by_volume.csv and stats_out_simple/top50_quotes_by_rows.csv …

Merged:
shape: (11, 7)
┌──────────┬────────────┬─────────────┬────────────┬────────────────┬──────────────────┬─────────────┐
│ SYM_ROOT ┆ SYM_SUFFIX ┆ TRADES_ROWS ┆ TRADES_VOL ┆ SYM_ROOT_right ┆ SYM_SUFFIX_right ┆ QUOTES_ROWS │
│ ---      ┆ ---        ┆ ---         ┆ ---        ┆ ---            ┆ ---              ┆ ---         │
│ str      ┆ str        ┆ i64         ┆ i64        ┆ str            ┆ str              ┆ i64         │
╞══════════╪════════════╪═════════════╪════════════╪════════════════╪══════════════════╪═════════════╡
│ TSLA     ┆            ┆ 1185389     ┆ 159068358  ┆ TSLA           ┆                  ┆ 8847235     │
│ SPY      ┆            ┆ 669221      ┆ 159259628  ┆ SPY            ┆                  ┆ 22131374    │
│ AAPL     ┆            ┆ 625486      ┆ 183784854  ┆ AAPL           ┆                  ┆ 9129733     │
│ AMZN     ┆            ┆ 492794      ┆ 156092695  ┆

/tmp/ipykernel_1305903/1662951033.py:17: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  .join(quotes_df, on=["SYM_ROOT", "SYM_SUFFIX"], how="outer")


In [34]:
# top50_trades_corresponding_quotes.py
'''
   • TRADES saved rows total: 9,324,631
   • QUOTES saved rows total: 128,520,108
   • Summary CSV: /home/amazon/gitcode/WithAmitG/TAQCode/stats_out_simple/merged_top50_sorted.csv
'''
'''
OUT_DIR = Path("./stats_out_simple")
MERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"
print(f"Reading {MERGED_OUT_CSV} …")
merged_df = pl.read_csv(MERGED_OUT_CSV)
print(merged_df)
'''

'\nOUT_DIR = Path("./stats_out_simple")\nMERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"\nprint(f"Reading {MERGED_OUT_CSV} …")\nmerged_df = pl.read_csv(MERGED_OUT_CSV)\nprint(merged_df)\n'

In [37]:
#top50_trades_corresponding_quotes.py
#top50_trades_corresponding_quotes_persist.py
!du -hc ~/Documents/TAQData/merged_output_top50/merged_all.parquet
print('shows top 50 tickers TAQ is not big')

1.5G	/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet
1.5G	total
shows top 50 tickers TAQ is not big


In [54]:
#ms.py for microstructure
%run ms.py


🔎 Microstructure analysis for: UBER


/home/amazon/gitcode/WithAmitG/TAQCode/ms.py:141: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.


shape: (1, 19)
┌────────┬────────────┬─────────┬────────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ TICKER ┆ FWD_TRADES ┆ N_ROWS  ┆ TOTAL_SIZE ┆ VWAP      ┆ AVG_SPREAD ┆ AVG_DEPTH_ ┆ IS_MEAN_BP ┆ IS_MEDIAN_ ┆ IS_MIN_BPS ┆ IS_MAX_BPS ┆ MI_MEAN_BP ┆ MI_MEDIAN_ ┆ MI_MIN_BPS ┆ MI_MAX_BPS ┆ MR_MEAN_BP ┆ MR_MEDIAN_ ┆ MR_MIN_BPS ┆ MR_MAX_BPS │
│ ---    ┆ _FOR_MR    ┆ ---     ┆ ---        ┆ ---       ┆ _BPS       ┆ TOB        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        │
│ str    ┆ ---        ┆ u32     ┆ i32        ┆ f64       ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ f64        ┆ f64        ┆ ---        ┆ ---        ┆ f64        ┆ f64        ┆ ---        ┆ ---        ┆ f64        ┆ f64        │
│        ┆ i32       

In [55]:
summary_df

TICKER,FWD_TRADES_FOR_MR,N_ROWS,TOTAL_SIZE,VWAP,AVG_SPREAD_BPS,AVG_DEPTH_TOB,IS_MEAN_BPS,IS_MEDIAN_BPS,IS_MIN_BPS,IS_MAX_BPS,MI_MEAN_BPS,MI_MEDIAN_BPS,MI_MIN_BPS,MI_MAX_BPS,MR_MEAN_BPS,MR_MEDIAN_BPS,MR_MIN_BPS,MR_MAX_BPS
str,i32,u32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""UBER""",10,3686949,462606101,62.814859,2974.294627,13.073162,10.596262,1.1462e-12,-31200.0,31133.333333,-20.450063,0.0,-31470.0,31133.333333,-35.410508,-0.0,-31470.0,10000.0


In [56]:
metrics_lf.head(1).collect()

TS,SYM_ROOT,SYM_SUFFIX,PRICE,SIZE,BID,ASK,MID,SPREAD,DEPTH_TOB,BIDSIZ,ASKSIZ,SIDE,MID_LAG,ARR_MID,IS_BPS,MI_BPS,MID_FWD,MR_BPS
datetime[μs],str,str,f64,i32,f64,f64,f64,f64,i64,i64,i64,i32,f64,f64,f64,f64,f64,f64
2023-12-15 04:00:01.106514,"""UBER""","""""",62.53,5,62.07,62.9,62.485,0.83,2,1,1,1,null,62.485,7.201728,0.0,62.625,-22.405377


In [57]:
metrics_lf.filter(pl.col('IS_BPS') > 31133 ).select(pl.len()).collect().item() #seeing how many rows have this max IS 

3

In [ ]:
metrics_lf.filter(pl.col('IS_BPS') > 31133 ).collect()

In [62]:
metrics_lf.filter(pl.col('IS_BPS') > 31133 ).collect(streaming=True)

/tmp/ipykernel_1305903/207101809.py:1: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  metrics_lf.filter(pl.col('IS_BPS') > 31133 ).collect(streaming=True)


TS,SYM_ROOT,SYM_SUFFIX,PRICE,SIZE,BID,ASK,MID,SPREAD,DEPTH_TOB,BIDSIZ,ASKSIZ,SIDE,MID_LAG,ARR_MID,IS_BPS,MI_BPS,MID_FWD,MR_BPS
datetime[μs],str,str,f64,i32,f64,f64,f64,f64,i64,i64,i64,i32,f64,f64,f64,f64,f64,f64
2023-12-15 16:06:35.182679,"""UBER""","""""",61.7,4900,61.66,61.7,61.68,0.04,2,1,1,1,15.0,15.0,31133.333333,31120.0,61.685,-0.810636
2023-12-15 16:13:26.293585,"""UBER""","""""",61.7,100,61.66,61.7,61.68,0.04,2,1,1,1,15.0,15.0,31133.333333,31120.0,62.65,-157.263294
2023-12-15 16:13:26.294092,"""UBER""","""""",61.7,100,61.66,61.74,61.7,0.08,3,1,2,1,15.0,15.0,31133.333333,31133.333333,61.575,20.259319
